# 실습 1

##### '4가지 같은 세기의 음을 합성하여 만든 화음 데이터를 가장 잘 근사할 수 있는 함수를 만들어 최종적으로 MSE(=MEAN SQUARED ERROR) Function을 활용하여 성능을 측정하라.'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import IPython.display as ipd
import librosa
import librosa.display
import numpy as np

filename = "/content/drive/MyDrive/Colab Notebooks/assign02/sound.wav"
data,sample_rate = librosa.load(filename)
ipd.Audio(filename)

In [3]:
print('data :', data[:5])
print('data size : ', len(data))

data : [0.         0.02578735 0.05151367 0.0770874  0.10244751]
data size :  11025


In [4]:
cf = [0, 0, 0, 0 ,0]

def func(cf) :
    time = np.linspace(0, 0.5, len(data))
    sound = np.sum(cf[0]*np.sin(np.array( [i*time for i in cf[1:]] )), axis=0)
    
    return sound

In [5]:
def mse(func):
    error=np.abs(data-func())
    return np.mean(error)*1500

def test() :
    return func(cf)

print('func error : ', mse(test))

func error :  211.1571269132653


##### Gradient Descent(경사하강법)으로 비용 함수를 최소화 해서, 최종적으로 Minimum에 도달해야한다.

In [6]:
init_cf = [max(data)/4, 800, 1000, 1200, 1400] 

lr = 0.0001  # 학습률
n_iter = 1000  # 반복 횟수

tmp_cf = init_cf  # init_cf를 tmp_cf로 복사

def test_tmp():
    return func(tmp_cf)

for i in range(n_iter):
    grad = np.zeros_like(tmp_cf)
    for j in range(len(tmp_cf)):
        # 파라미터에 대한 편미분을 계산합니다.
        tmp_cf_plus = tmp_cf.copy()
        tmp_cf_plus[j] += 1e-5  # 아주 작은 값을 더해줍니다.
        tmp_cf_minus = tmp_cf.copy()
        tmp_cf_minus[j] -= 1e-5  # 아주 작은 값을 빼줍니다.
        grad[j] = (mse(lambda: func(tmp_cf_plus)) - mse(lambda: func(tmp_cf_minus))) / (2 * 1e-5)
    tmp_cf -= lr * grad  # 경사하강법 적용
    
    tmp_err = mse(test_tmp)
    if i % 100 == 0:  # 100번째 마다 오차 출력
        print(f"Iteration {i}, MSE: {tmp_err}")

if tmp_err <= 5 :
    print("pass the test")    
else:
    print("fail the test")

Iteration 0, MSE: 214.3245588275141
Iteration 100, MSE: 211.01483008843178
Iteration 200, MSE: 211.01469884723087
Iteration 300, MSE: 211.01456947829638
Iteration 400, MSE: 211.0144253157085
Iteration 500, MSE: 211.0142932697889
Iteration 600, MSE: 211.01416152211434
Iteration 700, MSE: 211.0140308971848
Iteration 800, MSE: 211.0138982942532
Iteration 900, MSE: 211.013765301428
fail the test


# 실습 2

##### 콘크리트 압축 강도를 구하는 Linear Regression Model을 만들어 Gradient descent와 Normal Equation을 활용하여 두 방식 간의 차이를 이해한다.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# data load

import pandas as pd

pd_data=pd.read_excel("/content/drive/MyDrive/Colab Notebooks/assign02/Concrete_Data.xls")

print(pd_data.shape)
pd_data.describe()

(1030, 9)


,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.165631,73.895485,54.187136,181.566359,6.203112,972.918592,773.578883,45.662136,35.817836
std,104.507142,86.279104,63.996469,21.355567,5.973492,77.753818,80.175427,63.169912,16.705679
min,102.000000,0.000000,0.000000,121.750000,0.000000,801.000000,594.000000,1.000000,2.331808
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.707115
50%,272.900000,22.000000,0.000000,185.000000,6.350000,968.000000,779.510000,28.000000,34.442774
75%,350.000000,142.950000,118.270000,192.000000,10.160000,1029.400000,824.000000,56.000000,46.136287
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.599225


In [9]:
# normalization

normalization_data = (pd_data - pd_data.min())/(pd_data.max()-pd_data.min())
normalization_data.describe()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,0.409054,0.205608,0.270800,0.477576,0.192643,0.499763,0.450524,0.122698,0.417181
std,0.238601,0.240064,0.319822,0.170504,0.185512,0.226029,0.201143,0.173544,0.208125
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.206336,0.000000,0.000000,0.344511,0.000000,0.380814,0.343578,0.016484,0.266301
50%,0.390183,0.061213,0.000000,0.504990,0.197205,0.485465,0.465404,0.074176,0.400050
75%,0.566210,0.397746,0.591054,0.560878,0.315528,0.663953,0.577020,0.151099,0.545732
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
np_data = normalization_data.to_numpy()
X,y= np_data[:,:8], np_data[:,8:]

print(X.shape)
print(X[0].shape)
print(y.shape)

(1030, 8)
(8,)
(1030, 1)


In [11]:
X = np.append(X, np.ones((len(X), 1)), axis=1) # bias 추가
X.shape

(1030, 9)

### gradient descent : 경사하강법

In [12]:
w = np.zeros((9, 1)) # w(가중치)와 bias -> w -= lr * grad를 위해 변경
print(w.shape)

(9, 1)


In [13]:
def mse(w, X, y) :
    preds = X.dot(w) # X(입력값)과 w(가중치)의 내적 계산으로 예측값 구하기
    preds = preds.reshape(len(preds), -1) # 예측값 reshape
    loss = 0.5*np.sum((preds-y)**2)/len(X) # mse
    return loss

err = mse(w, X, y)
err

0.10865696458517649

In [14]:
lr = 0.0001 # 학습률 (Learning Rate) -> 반복 횟수를 위해 학습률을 줄였다.
step = 0 # 반복 횟수 (step)

while (err > 0.03) :
  pred = X.dot(w) # 예측값 계산
  error = pred - y # 예측오차 계산
  gradient = X.T.dot(error) # 예측오차를 이용해 gradient(기울기) 계산
  gradient = gradient.reshape(-1, 1) # 가중치 업데이트를 위한 기울기 reshape
  w -= lr * gradient # w(가중치) 업데이트
  err = mse(w, X, y) # mse 계산
  step += 1
  if step % 10 == 0 :
      print("[%d step] err : %f" % (step, err))
      print("current weight : ", w)
    
print('\ntotal step : ', step)
print("weight: ", w)
print("err : ", err)


total step :  5
weight:  [[0.07055811]
 [0.03312024]
 [0.03470828]
 [0.06277748]
 [0.03451712]
 [0.06675453]
 [0.06042389]
 [0.02342508]
 [0.14217345]]
err :  0.02923336413241906


### Normal Equation

In [15]:
mse(w, X, y)

coeffs = np.dot(X.T, X) # (X.T와 X) 행렬의 곱
coeffs_inv = np.linalg.inv(coeffs) # coeffs을 역으로 변환
coeffs = np.dot(coeffs_inv, X.T) # (역으로 변환한 값와 X.T)의 곱
coeffs = np.dot(coeffs, y) # 결과 값과 y의 곱
w = coeffs
err = mse(w, X, y)
print("weight : ", w) # Normal Equation으로 구한 w값 출력
print("err : ", err)

weight :  [[ 0.65363934]
 [ 0.46497947]
 [ 0.21923479]
 [-0.2345262 ]
 [ 0.1166117 ]
 [ 0.07727149]
 [ 0.10008502]
 [ 0.51799506]
 [-0.06289088]]
err :  0.008320205020961079


Gradient Descent에 비해 Normal Equation이 더 낮은 오차율을 보인다. 이러한 이유는 Normal Equation은 정형화된 식으로 보다 빠르게 정확한 값을 구해내지만 Gradient Descent는 계속해서 값을 갱신해 나가야 하므로 부정확할 확률이 높아진다.